In [ ]:
#Importo librerias a utilizar
import pandas as pd
import numpy as np
import ast
from datetime import datetime
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Visualizo el formato del dataframe con los 5 primeros registros
df = pd.read_csv('df_limpio.csv')
df.head(5)

In [ ]:
print(f"El dataframe contiene {df.shape[0]} columnas y {df.shape[1]} filas")

In [ ]:
#Visualizo los valores faltantes de las columnas del dataframe con missingno
msno.bar(df)
plt.show()

In [ ]:
dfML = pd.read_csv("df_limpio.csv")
dfML.info()

In [ ]:
#Creo columnas de MESES y DIAS para utilizar en visualizaciones

dfML["release_date"] = pd.to_datetime(dfML["release_date"])
dfML["release_month"] = dfML["release_date"].dt.month
dfML["release_day"] = dfML["release_date"].dt.day_name()

dias = {"Monday": "Lunes",
    "Tuesday": "Martes",
    "Wednesday": "Miercoles",
    "Thursday" : "Jueves",
    "Friday": "Viernes",
    "Saturday":"Sabado",
    "Sunday": "Domingo"}

dfML["release_day"] = dfML["release_day"].map(dias)

In [ ]:
#Grafico la cantidad de peliculas estrenadas en cada dia
conteo_dias = dfML["release_day"].value_counts().sort_index()
order = ["Domingo", "Lunes", "Martes", "Miercoles", "Jueves", "Viernes", "Sabado"]

sns.barplot(x=conteo_dias.index, y=conteo_dias.values, order=order)
plt.xlabel("Dia")
plt.ylabel("Cantidad de Registros")
plt.title("Cantidad de Registros por Mes")
plt.show()

In [ ]:
#Grafico la cantidad de peliculas estrenadas en cada mes
conteo_meses = dfML["release_month"].value_counts().sort_index()
sns.barplot(x=conteo_meses.index, y=conteo_meses.values)
plt.xlabel("Mes")
plt.ylabel("Cantidad de Registros")
plt.title("Cantidad de Registros por Mes")
plt.show()

In [ ]:
#Visualizo en un dataframe el director con mayor cantidad de peliculas y su retorno total
df_director = dfML.groupby(str("crew")).agg({'return': 'sum', 'title': 'count'})
df_director = df_director.sort_values('title', ascending=False)
df_director = df_director.reset_index() #La columna crew queda como indice, la devuelvo a su formato de columna con su nombre
df_director = df_director.rename(columns={'index': 'crew'})
df_director = df_director.drop(df_director[df_director['crew'] == "[]"].index) #elimino de la visualizacion las listas vacias en crew
df_director.head(10)

In [ ]:
#Visualizo en un dataframe los actores con mayor cantidad de peiculas y el promedio de votos
df_actores = df.explode('cast')
df_actores = dfML.groupby(("cast")).agg({'vote_average': 'mean', 'title': 'count'})
df_actores = df_actores.sort_values('title', ascending=False)
df_actores = df_actores.reset_index() #La columna cast queda como indice, la devuelvo a su formato de columna con su nombre
df_actores = df_actores.rename(columns={'index': 'cast'})
df_actores = df_actores.drop(df_actores[df_actores['cast'] == "[]"].index) #elimino de la visualizacion las listas vacias en cast
df_actores.head(10)

In [ ]:
#Visualizo las peliculas con mayor promedio de votos, con genero y actores
df_peliculas = dfML.groupby(("title")).agg({'vote_average': 'mean'})
df_peliculas = df_peliculas.sort_values('vote_average', ascending=False)
df_peliculas = df_peliculas.reset_index() #La columna cast queda como indice, la devuelvo a su formato de columna con su nombre
df_peliculas = df_peliculas.rename(columns={'index': 'title'})
df_peliculas["genres"] = dfML["genres"]
df_peliculas["cast"] = dfML["cast"]
df_peliculas

In [ ]:
#Visualizo los 10 paises con mayores producciones 

df_paises = dfML.explode('production_countries')
df_paises = dfML.groupby(("production_countries")).agg({'title': 'count'})
df_paises = df_paises.sort_values('title', ascending=False)
df_paises = df_paises.reset_index()
df_paises = df_paises.rename(columns={'index': 'production_countries'})
df_paises = df_paises.drop(df_paises[df_paises['production_countries'] == "[]"].index)
df_paises = df_paises[:10]
df_paises

In [ ]:
#Preparo las columnas title y overview para visualizar la nube de palabras de cada una
dfML["title"] = dfML["title"].astype("str")
dfML["overview"] = dfML["overview"].astype("str")
titulos = " ".join(dfML["title"])
overviews = " ".join(dfML["overview"])

In [ ]:
#Visualizo la nube de palabras mas destacadas en la columna titulos
wordcloudTitulos = WordCloud(stopwords=STOPWORDS, background_color="yellow", height=2000, width=2500).generate(titulos)
plt.figure(figsize=(16,8))
plt.imshow(wordcloudTitulos)
plt.axis("off")
plt.show()

In [ ]:
#Visualizo la nube de palabras mas destacadas en la columna overviews
wordcloudOverview = WordCloud(stopwords=STOPWORDS, background_color="yellow", height=2000, width=2500).generate(overviews)
plt.figure(figsize=(16,8))
plt.imshow(wordcloudOverview)
plt.axis("off")
plt.show()